In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms

#Librerias antes no vistas
from scipy import stats
from scipy.stats import norm, skew


pd.set_option('display.max_columns',None)


In [ ]:
train=pd.read_csv('https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/House_pricing/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/House_pricing/test.csv')

# Presentacion del dataset Housing Price California

In [ ]:
print('Para la data de train:', train.shape)
print('Para la data de test:',test.shape)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
ms.matrix(train)

In [ ]:
ms.matrix(test)

In [ ]:
train.drop("Id", axis = 1, inplace = True)
test.drop("Id",axis=1,inplace=True)

### Limpiando Nulos

- Nulos categoricos.

- Nulos numéricos.

In [ ]:
columnas_object_nulos = [col for col in test.columns if ((train[col].dtype =='object')|(test[col].dtype == 'object')&((train[col].isna().any()|(test[col].isna().any()))))]

In [ ]:
def reemplazando_non_nans(df, lista_cols_obj_nulos):
    for col in lista_cols_obj_nulos:
        df[col].fillna("None",inplace=True)

In [ ]:
reemplazando_non_nans(train,columnas_object_nulos)
reemplazando_non_nans(test,columnas_object_nulos)

In [ ]:
for col in ['BsmtFinSF2','BsmtFullBath','BsmtHalfBath','GarageCars']:
    train.loc[train[col].isna(),col] = 0.0
    test.loc[test[col].isna(),col] = 0

In [ ]:
lista_nulos_numericos = [col for col in test.columns if (test[col].isna().any())]

In [ ]:
for col in lista_nulos_numericos:
    plt.title(train[col].isna().sum())
    sns.distplot(train[col].dropna())
    plt.show()

In [ ]:
for col in lista_nulos_numericos:
    plt.title(test[col].isna().sum())
    sns.distplot(test[col].dropna())
    plt.show()

In [ ]:
for col in lista_nulos_numericos:
    train[col].fillna(train[col].mean(),inplace=True)
    test[col].fillna(test[col].mean(),inplace=True)

### Exportando a csv

In [ ]:
train.to_csv('train_limpio.csv',index=False)
test.to_csv('test_limpio.csv',index=False)

In [ ]:
train = pd.read_csv('train_limpio.csv')
summit = pd.read_csv('test_limpio.csv')

# Procesamiento de Datos

##### OUTLIERS

In [ ]:
sns.scatterplot(x = train['GrLivArea'], y = train['SalePrice'])

In [ ]:
sns.distplot(train['GrLivArea'])

In [ ]:
dict_outliers = {}
for col in train.columns:
    if train[col].dtype != 'object':        
        if train[col].kurt() > 20:
            dict_outliers[col] = train[col].kurt()
    else:
        pass

In [ ]:
for k,v in dict_outliers.items():
    sns.distplot(train[k])
    plt.show()

Se puede apreciar en la esquina inferior derecha dos datos que indican un **AreaHabitable** muy grande, y sin embargo registran un bajo precio. Estos valores se consideran outliers. Por ello procederemos a eliminarlos.

In [ ]:
#Eliminamos los outliers acorde a las variables del gráfico
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

#Verificamos el gráfico nuevamente para confirmar que los outliers hayan sido eliminados
sns.scatterplot(x = train['GrLivArea'], y = train['SalePrice'])

Hay probabilidades de que otros valores atipicos existan en los datos de entrenamiento (si es que alli hubieran tambien outliers), sin embargo removerlos todos puede afectar negativamente nuestros modelos. Por ello vamos a permitir aquellos en los cuales los modelos son lo suficientemente robustos.

## Variable Objetivo

**Precio** es la variable que vamos a predecir. Asi que vamos a hacer un poco de analisis en esta variable primero.

In [ ]:
from scipy.stats import norm, skew

In [ ]:
## Comparemos la distribución de Precio con una distribucion normal.
sns.distplot(train['SalePrice'], fit = norm)
plt.ylabel('Frecuencia')
plt.title('Distribución del Precio')

Se puede apreciar que los datos de la variable objetivo están inclinados a la izquierda. Sin embargo, los modelos lineales que veremos en la presente clase pueden optimizar su predición con datos distribuidos normalmente. Vamos a transformar los datos de esta variable para **aproximarlos a una distribución normal**.

 **Log-transformación de la variable objetivo**

In [ ]:
#Utilizamos la función log1p de la librería numpy que aplica la función log(1+x) a todos los elementos de la variable Precio
train["SalePrice"] = np.log1p(train["SalePrice"])

#Dibujamos la distribución actualizada de la variable Precio
sns.distplot(train['SalePrice'] , fit=norm);

plt.ylabel('Frequency')
plt.title('Distribución del Precio Real Ajustado')

**Vamos a revisar el archivo "DescripcionVariables.xlsx" el cual contiene un diccionario sobre el significado de las variables predictoras**

**Correlación de los Datos**


In [ ]:
#Mapa de correlación para ver cómo las variables predictoras están correlacionadas con Precio
corrmat = train.corr()
plt.subplots(figsize=(15,12))
sns.heatmap(corrmat, vmax=0.9, square=True)

# Feature Engineering

**Transformando algunas variables númericas que realmente son categóricas**

In [ ]:
#Cambiar las variables Año y Mes de venta de la casa a tipo entero
train['YrSold'] = train['YrSold'].astype(int)
train['MoSold'] = train['MoSold'].astype(int)

summit['YrSold'] = summit['YrSold'].astype(int)
summit['MoSold'] = summit['MoSold'].astype(int)

In [ ]:
#Añadiendo tamaño total del area (TotalBsmtSF + 1stFlrSF + 2ndFlrSF)


In [ ]:
#IfRemod será una columna dummy que nos ayudará a saber si la casa fue remodelada (1) o no (0)
train['IfRemod'] = np.where(train['YearBuilt'] == train['YearRemodAdd'], 0,1)
summit['IfRemod'] = np.where(summit['YearBuilt'] == summit['YearRemodAdd'], 0,1)

In [ ]:
# YRemodAftConst: Cuántos años tuvo de diferencia la casa remodelada con respecto a su año de construcción
train['YRemodAftConst'] = np.where(train['IfRemod'] == 1, train['YearRemodAdd'] - train['YearBuilt'], 0)
summit['YRemodAftConst'] = np.where(summit['IfRemod'] == 1, summit['YearRemodAdd'] - summit['YearBuilt'], 0)

**Detectamos el grado de asimetría de las variables predictoras** - Solo aquellas con data numérica

In [ ]:
#Filtramos aquellas variables que tengan datos diferentes al tipo object, es decir numéricas
numeric_feats = train.dtypes[train.dtypes != "object"].index

#Aplicamos la función skew a las variables filtradas y mostramos el resultado ordenando de forma ascendente
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSesgo en características numéricas: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

In [ ]:
skewness.Skew.unique()

### Tranformacion BoxCox

**Encodeamos rápidamente las variables categóricas**

In [ ]:
cat_variables = [col for col in train.columns if train[col].dtype == 'object']

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for cat in cat_variables:
    le.fit(list(train[cat].values)+list(summit[cat].values))
    train[cat] = le.transform(train[cat])
    summit[cat] = le.transform(summit[cat])

In [ ]:
train.columns

# Modelos de Regresión

**Importamos librerías**

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score

from sklearn.model_selection import train_test_split

In [ ]:
X = train.drop('SalePrice',axis=1)
y = train.SalePrice.values
X_train, X_test, y_train, y_test = train_test_split(X,y)

## Regresión Lineal

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
predicciones_lr = lr.predict(X_test)

In [ ]:
print('Mi Error cuadratico medio es de: {:.8f}'.format(mean_squared_error(y_test, predicciones_lr)))
print('Mi RMSE es de: {:.8f}'.format(np.sqrt(mean_squared_error(y_test,predicciones_lr))))

##  **Regresión Lasso**  : 

Este modelo es muy sensible a outliers, por ello necesitamos hacer más robusto el modelo a través del parámetro **Robustscaler** dentro de un pipeline

In [ ]:
lasso = Lasso(alpha=0.0005, random_state=1)
lasso.fit(X_train, y_train)
predicciones_lasso = lasso.predict(X_test)

In [ ]:
print('Mi Error cuadratico medio es de: {:.8f}'.format(mean_squared_error(y_test, predicciones_lasso)))
print('Mi RMSE es de: {:.8f}'.format(np.sqrt(mean_squared_error(y_test,predicciones_lasso))))

## Regresión Ridge:

In [ ]:
ridge = Ridge(alpha=1, solver = 'cholesky')
ridge.fit(X_train,y_train)
predicciones_ridge = ridge.predict(X_test)

In [ ]:
print('Mi Error cuadratico medio es de: {:.8f}'.format(mean_squared_error(y_test, predicciones_ridge)))
print('Mi RMSE es de: {:.8f}'.format(np.sqrt(mean_squared_error(y_test,predicciones_ridge))))

## **Regresión ElasticNet** :

In [ ]:
elasticnet = ElasticNet(alpha=0.0005,l1_ratio=0.5)
elasticnet.fit(X_train, y_train)
predicciones_elasticnet = elasticnet.predict(X_test)

In [ ]:
print('Mi Error cuadratico medio es de: {:.8f}'.format(mean_squared_error(y_test, predicciones_elasticnet)))
print('Mi RMSE es de: {:.8f}'.format(np.sqrt(mean_squared_error(y_test,predicciones_elasticnet))))

## Descenso estocástico de la Gradiente

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_r = SGDRegressor(max_iter = 1200000, penalty='elasticnet', eta0=0.01, tol=1e-3)
sgd_r.fit(X_train,y_train)

In [ ]:
predicciones_sgd = sgd_r.predict(X_test)

In [ ]:
print('Mi Error cuadratico medio es de: {:.8f}'.format(mean_squared_error(y_test, predicciones_sgd)))
print('Mi RMSE es de: {:.8f}'.format(np.sqrt(mean_squared_error(y_test,predicciones_sgd))))

## Regresión con Random Forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor(n_jobs=-1,max_depth=11,n_estimators=1500)
rf.fit(X_train,y_train)

In [ ]:
predicciones_rf = rf.predict(X_test)

In [ ]:
print('Mi Error cuadratico medio es de: {:.8f}'.format(mean_squared_error(y_test, predicciones_rf)))
print('Mi RMSE es de: {:.8f}'.format(np.sqrt(mean_squared_error(y_test,predicciones_rf))))

## Pipelines!


## Mejorando y evaluando los modelos

In [ ]:
#Creamos la función rmsle_cv para aplicar cross_val_score
def rmsle_cv(model,n_folds = 5):
    kf = KFold(n_folds, shuffle=True, random_state=1).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
lasso_pipeline = make_pipeline(RobustScaler(), Lasso(alpha=0.0005,random_state=1))
score_lasso = rmsle_cv(lasso_pipeline)
print("\nLasso score: {:.8f} ({:.8f})\n".format(score.mean(), score.std()))

In [ ]:
rf_pipeline = make_pipeline(RobustScaler(),RandomForestRegressor(n_jobs=-1,max_depth=11,n_estimators=1500))
score_rf = rmsle_cv(rf_pipeline)

print("\RandomForestRegressor score: {:.8f} ({:.8f})\n".format(score.mean(), score.std()))

## Evaluando nuestro mejor modelo

In [ ]:
#Mis mejores predicciones son de lasso sin pipeline (y sin transformacion)
plt.figure(figsize=(12,8))
plt.xlabel('Predicciones Lasso')
plt.ylabel('Data de verdad')
sns.regplot(x=predicciones_lasso,y=y_test)

In [ ]:
plt.figure(figsize=(12,8))
sns.regplot(x=predicciones_elasticnet,y=y_test)
plt.show()
plt.figure(figsize=(12,8))
sns.regplot(x=predicciones_ridge,y=y_test)
plt.show()

In [ ]:
# A ver ahora plotea el performance del Descenso Estocástico y del Random Forest...
# NO ES RANDOM FOREST un Método muy lejano????



#### La técnica blending (predecir en conjunto)

- ¿Cómo varía nuestro performance si para lasso, ridge y elasticnet  hacemos un promedio de las predicciones?

TAREA: 

Predigan para cada columna, sumen y promedien esa suma.
Luego saquen el MSE de las predicciones sumadas